# Setup

In [1]:
CASSANDRA_START_FROM_SCRATCH = True
DOCKER_INTERNAL_HOST = "host.docker.internal"
DOCKER_DNS = ["10.15.20.1"]

CASSANDRA_CLUSTER_NAME = "cassandra-cluster.mavasbel.vpn.itam.mx"
CASSANDRA_TOTAL_NODES = 3

CASSANDRA_NODE_IPS = ["10.15.20.2"] * CASSANDRA_TOTAL_NODES
CASSANDRA_NODE_NAMES = [f"cassandra-node-{i+1}" for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_HOSTNAMES = [
    f"{CASSANDRA_NODE_NAMES[i]}.mavasbel.vpn.itam.mx"
    for i in range(CASSANDRA_TOTAL_NODES)
]
CASSANDRA_NODE_GOSSIP_PORTS = [7000 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_RPC_PORTS = [9040 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_SSL_GOSSIP_PORTS = [7500 + (i + 1) for i in range(CASSANDRA_TOTAL_NODES)]
CASSANDRA_NODE_JMX_PORTS = [7200 + (i + 1) for i in range(0, CASSANDRA_TOTAL_NODES)]

CASSANDRA_CA_CERT_PASSWORD = "cassandra_cluster_ca_cert_passowrd"
CASSANDRA_NODE_CERT_PASSWORD = "cassandra_cluster_cert_passowrd"
CASSANDRA_INIT_USER = "cassandra"
CASSANDRA_INIT_PASSWORD = "cassandra"

CASSANDRA_WORKDIR = "/var/lib/cassandra"

In [2]:
import os
from pathlib import Path

LOCALHOST_WORKDIR = f"{os.path.join(os.path.relpath(Path.cwd()))}"
DOCKER_MOUNTDIR = os.path.join(LOCALHOST_WORKDIR, "mount")
CASSANDRA_LOCALHOST_CLUSTER_CA_CERTDIR = os.path.join(LOCALHOST_WORKDIR, "cluster_certs")

mount_path = Path(DOCKER_MOUNTDIR)
mount_path.mkdir(parents=True, exist_ok=True)
os.environ.setdefault("CQLENG_ALLOW_SCHEMA_MANAGEMENT", "True")

'True'

# Session creation

In [3]:
from cassandra.cluster import Cluster
from cassandra.connection import DefaultEndPoint
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import dict_factory

cassandra_nodes = [
    DefaultEndPoint(CASSANDRA_NODE_HOSTNAMES[j], CASSANDRA_NODE_RPC_PORTS[j])
    for j in range(CASSANDRA_TOTAL_NODES)
]
print(
    f"🔗 Connecting to: {[f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes]}"
)
print(
    f"JDBC URL: jdbc:cassandra://{','.join([f"{cassandra_node.address}:{cassandra_node.port}" for cassandra_node in cassandra_nodes])}"
)

auth_provider = PlainTextAuthProvider(
    username=CASSANDRA_INIT_USER, password=CASSANDRA_INIT_PASSWORD
)
cluster = Cluster(contact_points=cassandra_nodes, auth_provider=auth_provider)

session = cluster.connect()
session.row_factory = dict_factory
print(f"✅ Connected to cluster: {cluster.metadata.cluster_name}")
print(f"🌐 Nodes found: {len(cluster.metadata.all_hosts())}")

🔗 Connecting to: ['cassandra-node-1.mavasbel.vpn.itam.mx:9041', 'cassandra-node-2.mavasbel.vpn.itam.mx:9042', 'cassandra-node-3.mavasbel.vpn.itam.mx:9043']
JDBC URL: jdbc:cassandra://cassandra-node-1.mavasbel.vpn.itam.mx:9041,cassandra-node-2.mavasbel.vpn.itam.mx:9042,cassandra-node-3.mavasbel.vpn.itam.mx:9043
✅ Connected to cluster: cassandra-cluster.mavasbel.vpn.itam.mx
🌐 Nodes found: 3


In [4]:
import pprint
import pandas as pd
from IPython.display import display

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)
pd.options.display.html.use_mathjax = True
pd.options.display.html.border = 1
pd.options.display.html.table_schema = False
pd.options.display.expand_frame_repr = True

display(
    pd.DataFrame(
        [cassandra_host.__dict__ for cassandra_host in cluster.metadata.all_hosts()]
    )
    .sort_index()
    .transpose()
)

,0,1,2
endpoint,cassandra-node-3.mavasbel.vpn.itam.mx:9043,10.15.20.2:9041,10.15.20.2:9042
conviction_policy,<cassandra.policies.SimpleConvictionPolicy object at 0x0000014A34785A90>,<cassandra.policies.SimpleConvictionPolicy object at 0x0000014A3275D480>,<cassandra.policies.SimpleConvictionPolicy object at 0x0000014A3275D810>
host_id,e4fce9f8-021e-4a5b-afd0-00086598c543,26bf5bd8-6fbc-4437-846f-bfe743b7d50c,37f6c544-8d4e-44e7-9b71-19c8747c28c6
_datacenter,dc1,dc1,dc1
_rack,rack1,rack1,rack1
lock,<unlocked _thread.RLock object owner=0 count=0 at 0x0000014A347939C0>,<unlocked _thread.RLock object owner=0 count=0 at 0x0000014A325A0040>,<unlocked _thread.RLock object owner=0 count=0 at 0x0000014A32612A80>
is_up,True,True,True
listen_address,172.19.0.4,NaN,NaN
listen_port,7003.0,NaN,NaN
broadcast_address,10.15.20.2,10.15.20.2,10.15.20.2


In [5]:
# from cassandra.cluster import ResultSet

# peers_v2: ResultSet = session.execute("SELECT * FROM system.peers_v2")
# locals: ResultSet = session.execute("SELECT * FROM system.local")

# for peer_v2 in peers_v2:
#     print(" | ".join([f"{col}: {peer_v2[col]}" for col in peers_v2.column_names]))
# print("=" * 100)
# for local in locals:
#     print(" | ".join([f"{col}: {local[col]}" for col in locals.column_names]))
# print("=" * 100)

### Keyspace

In [6]:
keyspace_name = "generic_analytics"
if CASSANDRA_START_FROM_SCRATCH:
    session.execute(f"DROP KEYSPACE IF EXISTS {keyspace_name}")

In [7]:
keyspace_name = "generic_analytics"
session.execute(
    f"""
CREATE KEYSPACE IF NOT EXISTS {keyspace_name} 
WITH replication = {{
    'class': 'NetworkTopologyStrategy', 
    'dc1': {CASSANDRA_TOTAL_NODES}
}}
"""
)
session.set_keyspace(keyspace_name)

# Create Table
# Partition Key: city (distributes data)
# Clustering Column: user_id (sorts data within city)
session.execute(
    """
CREATE TABLE IF NOT EXISTS user_metrics (
    city text,
    user_id uuid,
    username text,
    session_duration int,
    last_access timestamp,
    PRIMARY KEY (city, user_id)
);
"""
)

### Insert

In [8]:
from faker import Faker

fake = Faker()

In [9]:
import uuid
from datetime import datetime
from cassandra.query import SimpleStatement
from cassandra import ConsistencyLevel
from cassandra.query import BatchStatement, BatchType

# 1. Prepare your statement outside the loop
query = """
INSERT INTO user_metrics (city, user_id, username, session_duration, last_access)
VALUES (?, ?, ?, ?, ?)
"""
prepared = session.prepare(query)
# prepared.consistency_level = ConsistencyLevel.QUORUM
# statement = SimpleStatement(query, consistency_level=ConsistencyLevel.QUORUM)

# 2. Create the Batch object
# LOGGED ensures atomicity but adds disk overhead, UNLOGGED is faster
batch_records = 200
batch = BatchStatement(batch_type=BatchType.LOGGED)
batch.consistency_level = ConsistencyLevel.QUORUM
print(f"Preparing batch of {batch_records} records...")
for _ in range(batch_records):
    batch.add(
        prepared,
        (
            fake.city(),
            uuid.uuid4(),
            fake.user_name(),
            fake.random_int(min=1, max=3600),
            datetime.now(),
        ),
    )

# 3. Execute the entire batch at once
session.execute(batch)
print("Batch successfully committed to the cluster.")

Preparing batch of 200 records...
Batch successfully committed to the cluster.


### Query

In [10]:
from cassandra.cluster import ResultSet
from typing import cast

print(cast(ResultSet, session.execute("SELECT count(*) FROM user_metrics")).one())

{'count': 200}


In [11]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT * FROM user_metrics LIMIT 100")
for row in rows.current_rows:
    print(row)

{'city': 'New Darin', 'user_id': UUID('fd389c99-add0-4f2e-bdf0-a9eec61e185a'), 'last_access': datetime.datetime(2026, 1, 25, 4, 12, 37, 548000), 'session_duration': 1750, 'username': 'ssmith'}
{'city': 'Port Dustin', 'user_id': UUID('3dd32a6a-d0ab-4b81-9a3f-878d48674888'), 'last_access': datetime.datetime(2026, 1, 25, 4, 12, 37, 547000), 'session_duration': 385, 'username': 'craiglee'}
{'city': 'Stephaniebury', 'user_id': UUID('b60ce87f-c1d4-43f2-a06b-f591d8cb4c10'), 'last_access': datetime.datetime(2026, 1, 25, 4, 12, 37, 536000), 'session_duration': 1845, 'username': 'natalie64'}
{'city': 'South Katie', 'user_id': UUID('e2536531-ae49-4f51-85c6-8c5d427b9348'), 'last_access': datetime.datetime(2026, 1, 25, 4, 12, 37, 531000), 'session_duration': 1255, 'username': 'williamlopez'}
{'city': 'East Travis', 'user_id': UUID('aa486205-a430-43bb-ba2b-8486420aa56d'), 'last_access': datetime.datetime(2026, 1, 25, 4, 12, 37, 511000), 'session_duration': 869, 'username': 'jason89'}
{'city': 'Port 

In [12]:
from cassandra.cluster import ResultSet

rows: ResultSet = session.execute("SELECT DISTINCT city FROM user_metrics", [])
for row in rows.current_rows:
    print(row)

{'city': 'New Darin'}
{'city': 'Port Dustin'}
{'city': 'Stephaniebury'}
{'city': 'South Katie'}
{'city': 'East Travis'}
{'city': 'Port Sara'}
{'city': 'Grossside'}
{'city': 'Mannmouth'}
{'city': 'Samuelbury'}
{'city': 'Collinston'}
{'city': 'New Rebeccaberg'}
{'city': 'New Kevin'}
{'city': 'South Dawn'}
{'city': 'Garybury'}
{'city': 'New Austinport'}
{'city': 'Port Gavin'}
{'city': 'Travistown'}
{'city': 'Port Leslie'}
{'city': 'South Allison'}
{'city': 'Port Richardmouth'}
{'city': 'Olsonville'}
{'city': 'Lake Carolshire'}
{'city': 'New Jacqueline'}
{'city': 'Rebeccaville'}
{'city': 'Shawland'}
{'city': 'Amandaside'}
{'city': 'East Robertville'}
{'city': 'South Denise'}
{'city': 'Webbhaven'}
{'city': 'New Davidfurt'}
{'city': 'Port Stevenmouth'}
{'city': 'Port Thomas'}
{'city': 'North Robertabury'}
{'city': 'Tonyaville'}
{'city': 'New Audreyport'}
{'city': 'New Jerry'}
{'city': 'Karenland'}
{'city': 'East Jenniferfort'}
{'city': 'Matthewhaven'}
{'city': 'Lake Gregory'}
{'city': 'West 

### Find nodes storing random data

In [13]:
import random
import pprint

random_token = random.randint(-9223372036854775808, 9223372036854775807)
query = "SELECT * FROM user_metrics WHERE token(city) >= %s LIMIT 1"
row: dict = cast(ResultSet, session.execute(query, [random_token])).one()
print(f"Random user_metric: {pprint.pformat(row)}")

prepared = session.prepare("SELECT * FROM user_metrics WHERE city=? AND user_id=?")
bound = prepared.bind([row['city'], row['user_id']])
routing_key = bound.routing_key
nodes = cluster.metadata.get_replicas(keyspace_name, routing_key)

print(f"Nodes storing '{row['city']}':")
for node in nodes:
    print(f" - Host: {node.address}, Gossip Port: {node.broadcast_port}")

Random user_metric: {'city': 'Baileyborough',
 'last_access': datetime.datetime(2026, 1, 25, 4, 12, 37, 528000),
 'session_duration': 402,
 'user_id': UUID('72deaa1e-590a-4755-821f-1c20c8724186'),
 'username': 'madison43'}
Nodes storing 'Baileyborough':
 - Host: cassandra-node-3.mavasbel.vpn.itam.mx, Gossip Port: 7003
 - Host: 10.15.20.2, Gossip Port: 7002
 - Host: 10.15.20.2, Gossip Port: 7001


### ORM-like

In [14]:
from typing import cast
from cassandra.cqlengine import columns
from cassandra.cqlengine.query import ModelQuerySet
from cassandra.cqlengine.models import Model
from cassandra.cqlengine.management import sync_table, create_keyspace_network_topology
from cassandra.cqlengine import connection

# 1. Connect the engine
connection.set_session(session)


# 2. Define your "Generic" Model
class UserMetrics(Model):
    __table_name__ = "user_metrics"

    # FIRST primary_key=True is the Partition Key
    # city = columns.Text(primary_key=True)
    city = columns.Text(primary_key=True, partition_key=True)

    # SECOND primary_key=True is the Clustering Key
    user_id = columns.UUID(primary_key=True, default=uuid.uuid4)

    # Attributes (Data)
    username = columns.Text(index=True)
    session_duration = columns.Integer()
    last_access = columns.DateTime()


# 3. Create Keyspace and Sync Table (Equivalent to CREATE TABLE)
create_keyspace_network_topology(keyspace_name, {"dc1": CASSANDRA_TOTAL_NODES})
sync_table(UserMetrics)

In [15]:
# 4. Use it like an ORM
# Create UserMetrics
new_metric: UserMetrics = UserMetrics.create(
    city=fake.city(),
    username=fake.name(),
    session_duration=120,
    last_access=datetime.now(),
)
new_metric.save()
print(f"Saved user_metris: {new_metric}")


# 5. Query UserMetrics
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering() # Non-recommended, used for non-primary key queries
)
for user_metric in user_metrics:
    print(f"Query user_metris: {user_metric}")

Saved user_metris: UserMetrics <city=Jasonview, user_id=717762a3-0dde-4c2b-aa4f-beac2d9b5036>
Query user_metris: UserMetrics <city=New Darin, user_id=fd389c99-add0-4f2e-bdf0-a9eec61e185a>
Query user_metris: UserMetrics <city=Port Dustin, user_id=3dd32a6a-d0ab-4b81-9a3f-878d48674888>
Query user_metris: UserMetrics <city=Stephaniebury, user_id=b60ce87f-c1d4-43f2-a06b-f591d8cb4c10>
Query user_metris: UserMetrics <city=South Katie, user_id=e2536531-ae49-4f51-85c6-8c5d427b9348>
Query user_metris: UserMetrics <city=East Travis, user_id=aa486205-a430-43bb-ba2b-8486420aa56d>
Query user_metris: UserMetrics <city=Port Sara, user_id=090a0b90-5deb-457d-bbc0-90838f125066>
Query user_metris: UserMetrics <city=Grossside, user_id=f9323a6b-28df-474d-a776-c723c671cfd0>
Query user_metris: UserMetrics <city=Mannmouth, user_id=33de9710-651e-4fdd-82a4-2d78e78bebf9>
Query user_metris: UserMetrics <city=Samuelbury, user_id=0bc28590-cf0c-417a-bc71-7207e48aa4fb>
Query user_metris: UserMetrics <city=Collinston, 

### Insert/Create

In [16]:
from typing import cast

cast(ModelQuerySet, UserMetrics.ttl(86400)).create(
    city=fake.city(),
    username=fake.user_name(),
    session_duration=fake.random_int(10, 1000),
    last_access=fake.date_time(),
)

UserMetrics(city='South Carolyn', user_id=UUID('8cb0655f-082b-4a25-92a9-35553b1d3b8d'), username='lawrence29', session_duration=715, last_access=datetime.datetime(1980, 4, 7, 21, 29, 16))

In [17]:
from typing import cast
from cassandra.cqlengine.query import BatchQuery
from cassandra.cqlengine.query import ModelQuerySet

with BatchQuery() as b:
    for _ in range(100):
        cast(ModelQuerySet, UserMetrics.batch(b)).create(
            city=fake.city(),
            username=fake.user_name(),
            session_duration=fake.random_int(10, 1000),
            last_access=fake.date_time(),
        )

### Delete

In [18]:
user_metrics = (
    cast(ModelQuerySet, UserMetrics.objects())
    .filter(session_duration__gte=120)
    .allow_filtering() # Non-recommended, used for non-primary key queries
)
for user_metric in user_metrics:
    user_metric.delete()
    print(f"Delete user_metris: {user_metric}")

Delete user_metris: UserMetrics <city=Allisonland, user_id=af873ff6-2c66-48ec-b8da-420ed6882dc4>
Delete user_metris: UserMetrics <city=New Darin, user_id=fd389c99-add0-4f2e-bdf0-a9eec61e185a>
Delete user_metris: UserMetrics <city=Port Dustin, user_id=3dd32a6a-d0ab-4b81-9a3f-878d48674888>
Delete user_metris: UserMetrics <city=Stephaniebury, user_id=b60ce87f-c1d4-43f2-a06b-f591d8cb4c10>
Delete user_metris: UserMetrics <city=Orrhaven, user_id=61d685aa-5c6f-4b61-bac4-147a8db317e3>
Delete user_metris: UserMetrics <city=South Carolyn, user_id=8cb0655f-082b-4a25-92a9-35553b1d3b8d>
Delete user_metris: UserMetrics <city=Moralesbury, user_id=14a481d9-3abf-4552-bada-f444d248b0a1>
Delete user_metris: UserMetrics <city=Bakerfurt, user_id=b5155bc0-7e8c-421b-a6d8-f3715a29288d>
Delete user_metris: UserMetrics <city=South Katie, user_id=e2536531-ae49-4f51-85c6-8c5d427b9348>
Delete user_metris: UserMetrics <city=East Travis, user_id=aa486205-a430-43bb-ba2b-8486420aa56d>
Delete user_metris: UserMetrics <

In [19]:
# from cassandra.cqlengine import connection

# # This isn't a compaction, but it forces RAM data to disk so the background compaction process can see it.
# connection.execute(f"ALTER TABLE {keyspace_name}.user_metrics WITH gc_grace_seconds = 10")